# nbscholar 大牛学者

> nbscholar (new-big scholar, 大牛学者) is a extension for better NoteBook development, extending fastai's `nbdev` libary. It is designed to assist you become a New-Big (a.k.a. 牛逼 or awesome) scholar one day.

In [31]:
#| default_exp nbscholar

In [32]:
#| hide
from nbdev.showdoc import *

In [33]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
#| export
from fastcore.script import call_parse

In [35]:
#| export
functions = []

子模块

In [ ]:
# | export
import re
import subprocess
from pathlib import Path
from scholarly_infrastructure.logging.nucleus import logger
from fastcore.script import call_parse


@call_parse
def nbscholar_submodules_to_ssh(path: str = "."):
    """
    Convert all HTTPS submodule URLs in `.gitmodules` to SSH URLs (e.g. https://github.com/user/repo.git → git@github.com:user/repo.git)
    """

    gitmodules = Path(path) / ".gitmodules"
    if not gitmodules.exists():
        logger.error(
            "❌ 未找到 .gitmodules 文件，请在含有子模块的项目根目录运行此函数。"
        )
        raise FileNotFoundError(".gitmodules not found")

    content = gitmodules.read_text(encoding="utf-8")

    # 匹配 https://github.com/user/repo(.git)
    pattern = re.compile(r"https://([^/]+)/([^/]+)/([^/\n]+)(\.git)?")

    changed = False

    def https_to_ssh(match):
        nonlocal changed
        host, user, repo, dotgit = match.groups()
        ssh_url = f"git@{host}:{user}/{repo}.git"
        logger.info(f"🔁 转换: https://{host}/{user}/{repo} → {ssh_url}")
        changed = True
        return ssh_url

    new_content = pattern.sub(https_to_ssh, content)

    if changed:
        gitmodules.write_text(new_content, encoding="utf-8")
        logger.success("✅ 已更新 .gitmodules 文件中的 URL。")

        # 同步子模块配置
        res = subprocess.run(["git", "submodule", "sync", "--recursive"])
        if res.returncode == 0:
            logger.success("✅ 已执行 `git submodule sync --recursive`。")
        else:
            logger.warning("⚠️ `git submodule sync` 执行失败，请手动检查。")
    else:
        logger.info("ℹ️ 没有发现 HTTPS 格式的子模块 URL，无需修改。")

In [ ]:
# | export
@call_parse
def nbscholar_submodules(mode: str = "to_ssh", *args, **kwargs):
    if mode == "to_ssh":
        nbscholar_submodules_to_ssh(*args, **kwargs)
    else:
        raise ValueError(f"Unknown mode: {mode}")


functions.append(nbscholar_submodules)



## nbscholar_docs

alternative to nbdev_docs

https://github.com/AnswerDotAI/nbdev/issues/1464





## nbscholar_license

use google addlicense to add license to all files in the repo, while ignoring gitignore and gitmodules.
 

## nbscholar_check

检查ipynb文件是否有语法错误，并且定位错误的位置。
由于 nbdev_export 报错的时候无法定位具体是哪个笔记本出错，所以我们的这个工具非常有用。

 

In [ ]:
# | export
import json
import ast
import re
import os
import argparse
from pathlib import Path
from scholarly_infrastructure.logging.nucleus import logger

def is_export_cell(code: str) -> bool:
    """
    Return True if the first non-empty line starts with '#| export' or '#| exporti'.
    """
    lines = code.splitlines()
    first_idx = next((i for i, l in enumerate(lines) if l.strip()), None)
    if first_idx is None:
        return False
    first = lines[first_idx].strip()
    return re.match(r'^#\s*\|\s*exporti?\b', first) is not None


def _normalize_ipython_code(code: str):
    """
    Normalize IPython syntax so we can safely run ast.parse.
    Returns:
      - str: normalized Python code
      - None: skip this cell (non-Python cell magic like %%bash/%%html/%%markdown/etc.)
    """
    lines = code.splitlines()

    # Find first non-empty line
    first_idx = next((i for i, l in enumerate(lines) if l.strip()), None)
    if first_idx is None:
        return ""

    first = lines[first_idx].lstrip()
    if first.startswith("%%"):
        magic = (first[2:].strip().split()[0].lower() if len(first) > 2 else "")
        non_python_magics = {
            "bash", "sh", "html", "markdown", "js", "javascript",
            "latex", "sql", "ruby", "perl", "svg", "writefile", "cython"
        }
        # Non-Python cell magic: skip the whole cell from syntax checking
        if magic in non_python_magics or magic == "":
            return None
        # Python-related cell magic: drop the magic line, keep the rest
        lines.pop(first_idx)

    def is_line_magic(s: str) -> bool:
        return s.lstrip().startswith("%")

    def is_introspection_line(s: str) -> bool:
        t = s.strip()
        return t.startswith("?") or t.endswith("?") or t.endswith("??")

    filtered = [
        ln for ln in lines
        if not is_line_magic(ln) and not is_introspection_line(ln)
    ]
    return "\n".join(filtered)


def check_ipynb_file(filepath):
    """
    Check a single ipynb file for Python syntax errors in code cells.
    For cells starting with '#| export' or '#| exporti', strictly enforce Python AST (no IPython syntax allowed).
    For normal cells, allow IPython syntax (%%, %, ?/??) by normalizing or skipping those lines.
    Returns True if no errors, False otherwise.
    """
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            notebook = json.load(f)
    except json.JSONDecodeError as e:
        logger.error(f"❌ {filepath} - Invalid JSON: {e}")
        return False

    if "cells" not in notebook:
        logger.warning(f"⚠️  {filepath} - No 'cells' key found (malformed notebook)")
        return True  # Not a syntax error in code, skip

    errors_found = False

    for cell_idx, cell in enumerate(notebook["cells"], start=1):
        if cell.get("cell_type") != "code":
            continue  # Skip non-code cells

        raw_code = "".join(cell.get("source", []))
        if not raw_code.strip():
            continue  # Skip empty code cells

        # Export cells: strict AST on raw code; Non-export: normalized AST
        if is_export_cell(raw_code):
            normalized_code = raw_code
        else:
            normalized_code = _normalize_ipython_code(raw_code)

        # Skip non-Python IPython cells (e.g., %%bash)
        if normalized_code is None or not normalized_code.strip():
            continue

        try:
            ast.parse(normalized_code)
        except SyntaxError as e:
            abs_path = str(Path(filepath).resolve())
            line = e.lineno or 1
            col = e.offset or 0
            # VSCode 友好格式：/abs/path/to/file:line:column: message
            logger.error(f"{abs_path}:{line}:{col}: SyntaxError: {e.msg} (cell {cell_idx})")
            if e.text:
                line_text = e.text.rstrip()
                logger.error(line_text)
                if col > 0:
                    logger.error(" " * (col - 1) + "^")
            errors_found = True

    if not errors_found:
        logger.info(f"✅ {filepath} - No syntax errors found")
    return not errors_found

In [36]:
# | export

from scholarly_infrastructure.logging.nucleus import logger

def check_directory(dirpath):
    """
    Check all ipynb files in a directory (and subdirectories) for syntax errors.
    Returns True if all files are error-free, False otherwise.
    """
    all_ok = True
    file_count = 0
    error_count = 0

    dir_path = Path(dirpath)
    # 使用rglob递归查找所有ipynb文件
    for filepath in dir_path.rglob("*.ipynb"):
        file_count += 1
        if not check_ipynb_file(str(filepath)):
            error_count += 1    
            all_ok = False

    logger.info("\n--- Summary ---")
    logger.info(f"Checked {file_count} .ipynb files")
    logger.info(f"Found errors in {error_count} files")
    logger.info(f"All files syntax error-free: {'Yes' if all_ok else 'No'}")
    return all_ok

In [ ]:
# | export
from fastcore.script import call_parse
@call_parse
def nbscholar_check(
    target_path: str = ".",  # Path to .ipynb file or directory containing .ipynb files
):
    """
    Check .ipynb files for severe Python syntax errors (indent, missing colons/comma, etc.)
    """
    if not os.path.exists(target_path):
        raise FileNotFoundError(f"❌ Path does not exist: {target_path}")

    if os.path.isfile(target_path):
        if not target_path.endswith(".ipynb"):
            raise ValueError(f"❌ Not a .ipynb file: {target_path}")
        check_ipynb_file(target_path)
    else:
        check_directory(target_path)


functions.append(nbscholar_check)

NameError: name 'call_parse' is not defined

## nbscholar_export

alternative to nbdev_export

In [38]:
#| export
import configparser
import os
from pathlib import Path

In [ ]:
#| export
# from skinfra.experiment import load_overlaying_config
def read_settings_ini(directory:str, item="nbs_path", track="DEFAULT", ini_name='settings.ini'):
    # return load_overlaying_config(
    #     directory, ini_name
    # )[track][item]
    config = configparser.ConfigParser()
    # 逐级向上查找，直到找到第一个包含 settings.ini 的目录
    current = directory
    
    while True:
        settings_path = os.path.join(current, ini_name)
        if os.path.exists(settings_path):
            break
        parent = os.path.dirname(current)
        if parent == current:  # 已到达根目录
            raise FileNotFoundError(f"Could not find {ini_name} in any parent of {directory}")
        current = parent

    config.read(settings_path)
    assert track in config, f"Could not find {track} in {settings_path}"
    assert item in config[track], f"Could not find {item} in {settings_path}"
    return config[track][item]

In [ ]:
#| export
import subprocess
import os
from scholarly_infrastructure.logging.nucleus import logger
@call_parse
def nbscholar_export(path:str = "."):
    res = os.system("nbdev_export")
    if res!= 0:
        logger.error("nbdev_export failed, now I will use `nbscholar_check` to help you locate the error. ")
        nbscholar_check(path)
        raise Exception("nbdev_export failed")
    # 读取 settings.ini 的 lib_path
    lib_path = read_settings_ini(path, item="lib_path")

    # MKINIT 生成 __init__.py
    # res = os.system(f"mkinit {lib_name} -w --lazy_loader --recursive --relative")
    res = os.system(f"mkinit {lib_path} -w --lazy_loader_typed --recursive --relative")
    if res!= 0:
        # raise Exception("mkinit failed")
        logger.warning("mkinit failed")
    
    # RUFF 格式化
    res = os.system(f"ruff format {lib_path}")
    if res!= 0:
        logger.warning("ruff format failed")
functions.append(nbscholar_export)

    

## nbscholar_translate

https://github.com/AnswerDotAI/nbdev/issues/1429

## nbscholar_license

https://github.com/AnswerDotAI/nbdev/issues/1475

## nbscholar_separate

https://github.com/AnswerDotAI/nbdev/issues/1468

In [59]:
#| export
import os
import nbformat
import re
from nbformat.notebooknode import NotebookNode, from_dict
from scholarly_infrastructure import default_on_exception

In [60]:
#| export
read_settings_ini_none = default_on_exception(read_settings_ini, default_value=None)


def guess_notebooks_path(directory="."):
    if isinstance(directory, Path):
        directory = directory.as_posix()
    # 读取 setting.ini 的 nbs_path， 如果有的话，没有就返回None
    return read_settings_ini_none(directory)

In [61]:
from scholarly_infrastructure.help import runs_path

In [62]:
# 创建一个临时的 settings.ini 文件进行测试
with open(runs_path/"settings.ini", "w") as f:
    f.write("[DEFAULT]\nnbs_path=notebooks")

# 测试 guess_notebooks_path 函数
print(guess_notebooks_path(runs_path))  # 应该输出 'notebooks'

# 删除临时的 settings.ini 文件
os.remove(runs_path/"settings.ini")

# 测试 settings.ini 文件不存在的情况
print(guess_notebooks_path(runs_path))  # 应该输出 None

# 创建一个不包含 nbs_path 的 settings.ini 文件进行测试
with open(runs_path/"settings.ini", "w") as f:
    f.write("[DEFAULT]\nother_key=other_value")

# 测试 guess_notebooks_path 函数
print(guess_notebooks_path(runs_path))  # 应该输出 None

# 删除临时的 settings.ini 文件
os.remove(runs_path/"settings.ini")

notebooks
None
None


In [63]:
#| export
from copy import deepcopy

In [64]:
#| export
def split_import_and_code_cells(notebook, inplace=True):
    """
    Process a Jupyter Notebook file, splitting cells with both import and non-import lines into two cells.
    The first new cell will contain only import statements, and the second will contain the rest of the code.
    """
    notebook = notebook if inplace else deepcopy(notebook)
    
    new_cells = []

    for cell in notebook['cells']:
        if cell['cell_type'] == 'code':
            # Split the lines in the cell
            lines = cell['source'].splitlines()

            # Extract leading blank lines or lines starting with "#|"
            leading_lines = []
            while lines and (lines[0].strip() == "" or lines[0].startswith("#|")):
                leading_lines.append(lines.pop(0))

            # Separate import statements and other code lines
            import_lines = [line for line in lines if re.match(r"^\s*import\b|^\s*from\b", line)]
            other_lines = [line for line in lines if line not in import_lines]

            if import_lines and other_lines:
                # Add the leading lines to the import cell
                
                new_cells.append(from_dict(cell | {
                    "cell_type": "code",
                    "metadata": {},
                    "source": "\n".join(leading_lines + import_lines), 
                    "outputs": []
                }))
                # Add the leading lines to the other code cell
                new_cells.append(from_dict(cell | {
                    "cell_type": "code",
                    "metadata": {},
                    "source": "\n".join(leading_lines + other_lines),
                    "outputs": cell['outputs']
                }))
            else:
                # If no split is needed, retain the original cell
                new_cells.append(cell)
        else:
            # Retain non-code cells as is
            new_cells.append(cell)

    # Update the notebook with the modified cells
    notebook['cells'] = new_cells
    return notebook

In [65]:
#| export
def operate_on_notebook_in(input_path, output_path=None, operation=split_import_and_code_cells):
    if output_path is None:
        output_path = input_path
    with open(input_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    notebook = operation(notebook)
    # Save the modified notebook
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)

In [66]:
#| export
def process_notebooks_in_folder(folder_path, operation=split_import_and_code_cells):
    """
    Traverse all .ipynb files in a folder and apply the cell-splitting logic.
    """
    if isinstance(folder_path, Path):
        folder_path = folder_path.as_posix()
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.ipynb'):
                notebook_path = os.path.join(root, file)
                print(f"Processing {notebook_path}")
                operate_on_notebook_in(notebook_path, operation=operation)

In [67]:
#| export
@call_parse
def nbscholar_separate(path:str="."):
    if os.path.isfile(path):
        return operate_on_notebook_in(path, split_import_and_code_cells)
    
    notebook_path = guess_notebooks_path(path)
    if notebook_path is None:
        notebook_path = path
    process_notebooks_in_folder(notebook_path)
functions.append(nbscholar_separate)

In [68]:
#| export
functions_dict = {
    v.__name__.replace("nbscholar_", ""):v for v in functions
}

# 整体路由
@call_parse
def nbscholar(mode:str="export", *args, **kwargs):
    function = functions_dict.get(mode, None)
    if function is None:
        raise ValueError(f"Invalid mode: {mode}")
    function(*args, **kwargs)


In [69]:
#| hide
import nbdev; nbdev.nbdev_export()